## Hw1

### Import Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import os 
%matplotlib inline
np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

# import keras
import keras
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPool2D, BatchNormalization, SeparableConv2D, Activation
from keras.optimizers import RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from keras.layers import Input
from tensorflow.python.keras.models import Model
from keras.preprocessing import image
from keras.models import load_model
sns.set(style='white', context='notebook', palette='deep')

# import PIL
from PIL import Image
import glob

# import scipy
from scipy.signal import convolve2d

Using TensorFlow backend.


In [4]:
num_class = 13
input_size = 256
batch_size = 64

### load the Pretrain Model 

In [9]:
classifier = load_model('pretrain_model.h5')
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 8, 8, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8388864   
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)               

#### show Model Structure

In [10]:
for i, layer in enumerate(classifier.layers):
    if i == 0:
        for layer0 in classifier.layers[i].layers[:]:
            layer0.trainable = False
            print('-'*40)
            print('%-25s'%layer0.name, '%10s'%layer0.trainable)
    else:
        print('-'*40)
        print('%-25s'%layer.name, '%10s'%layer.trainable)

----------------------------------------
input_1                        False
----------------------------------------
block1_conv1                   False
----------------------------------------
block1_conv2                   False
----------------------------------------
block1_pool                    False
----------------------------------------
block2_conv1                   False
----------------------------------------
block2_conv2                   False
----------------------------------------
block2_pool                    False
----------------------------------------
block3_conv1                   False
----------------------------------------
block3_conv2                   False
----------------------------------------
block3_conv3                   False
----------------------------------------
block3_conv4                   False
----------------------------------------
block3_pool                    False
----------------------------------------
block4_conv1           

### Import Dataset & Preprocessing

In [11]:
def training_size(full_data=False, validation_split_ratio=0.2):
    if full_data:
        valid_split = 0.05
    else:
        valid_split = validation_split_ratio
        
    train_datagen = ImageDataGenerator(
        height_shift_range=0.2,
        width_shift_range=0.2,
        zoom_range=0.2,
        shear_range=0.2,
        validation_split=valid_split)

    valid_datagen = ImageDataGenerator(
        validation_split=valid_split)
    
    return train_datagen, valid_datagen
train_datagen, valid_datagen = training_size(full_data=True, validation_split_ratio=0.2)

In [12]:
training_set = train_datagen.flow_from_directory(
    'cs-ioc5008-hw1/train',
    target_size=(256, 256),
    subset='training',
    batch_size=batch_size,
    class_mode='categorical',
    seed=3023)

validation_set = valid_datagen.flow_from_directory(
    'cs-ioc5008-hw1/train',
    target_size=(256, 256),
    subset='validation',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=3023)

Found 2684 images belonging to 13 classes.
Found 135 images belonging to 13 classes.


In [13]:
label_dict = training_set.class_indices
label_dict

{'bedroom': 0,
 'coast': 1,
 'forest': 2,
 'highway': 3,
 'insidecity': 4,
 'kitchen': 5,
 'livingroom': 6,
 'mountain': 7,
 'office': 8,
 'opencountry': 9,
 'street': 10,
 'suburb': 11,
 'tallbuilding': 12}

### Build Model

In [14]:
optimizer = RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer = SGD(lr=0.0001, momentum=0.9)

classifier.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

#### save model when meets the highest validation set accuracy

In [15]:
ac = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

In [16]:
classifier.fit_generator(
    training_set,
    steps_per_epoch=45,
    epochs=20,
    validation_data=validation_set,
    validation_steps=32.5,
    verbose=1,
    callbacks = [ac]
)

Epoch 1/20
45/45 [==============================] - 72s 2s/step - loss: 2.0364 - accuracy: 0.6100 - val_loss: 0.0116 - val_accuracy: 0.8296

Epoch 00001: val_accuracy improved from -inf to 0.82963, saving model to pretrain_bs64_spe45.h5
Epoch 2/20
45/45 [==============================] - 59s 1s/step - loss: 0.3958 - accuracy: 0.8625 - val_loss: 0.0065 - val_accuracy: 0.8889

Epoch 00002: val_accuracy improved from 0.82963 to 0.88889, saving model to pretrain_bs64_spe45.h5
Epoch 3/20
45/45 [==============================] - 58s 1s/step - loss: 0.2865 - accuracy: 0.9148 - val_loss: 0.0046 - val_accuracy: 0.9630

Epoch 00003: val_accuracy improved from 0.88889 to 0.96296, saving model to pretrain_bs64_spe45.h5
Epoch 4/20
45/45 [==============================] - 57s 1s/step - loss: 0.2317 - accuracy: 0.9265 - val_loss: 0.0065 - val_accuracy: 0.9556

Epoch 00004: val_accuracy did not improve from 0.96296
Epoch 5/20
45/45 [==============================] - 59s 1s/step - loss: 0.1937 - accura

### Load the previously stored model to predict the test set

In [17]:
from keras.models import load_model
model = load_model("model.h5")

In [18]:
file_path = 'cs-ioc5008-hw1/test/'
file = sorted(glob.glob(file_path + '*.jpg'))
y_pred = []

for i in range(len(file)):
    images = image.load_img(file[i], target_size=(256, 256, 3))
    x = image.img_to_array(images)
    x = x.reshape([-1, 256, 256, 3])
    x = model.predict(x)
    x = np.argmax(x, axis = 1)
    y_pred.append(x[0])

In [19]:
path = os.listdir('cs-ioc5008-hw1/train/')
labels = sorted([p for p in path])
labels = dict(zip([i for i in range(len(labels))], labels))
labels

{0: 'bedroom',
 1: 'coast',
 2: 'forest',
 3: 'highway',
 4: 'insidecity',
 5: 'kitchen',
 6: 'livingroom',
 7: 'mountain',
 8: 'office',
 9: 'opencountry',
 10: 'street',
 11: 'suburb',
 12: 'tallbuilding'}

In [21]:
y_pred_tolabel = [labels[i] for i in y_pred]
submission = pd.read_csv('cs-ioc5008-hw1/sameple_submission.csv')
submission['label'] = y_pred_tolabel
submission

,id,label
0,image_0000,highway
1,image_0001,bedroom
2,image_0002,suburb
3,image_0003,bedroom
4,image_0004,tallbuilding
...,...,...
1035,image_1035,mountain
1036,image_1036,opencountry
1037,image_1037,coast
1038,image_1038,forest


#### output the prediction set and submit to kaggle

In [22]:
import csv
submission.to_csv('submission.csv', index=False)